# Pydantic AI Walkthrough

Inspired bij: https://ai.pydantic.dev/agents/

Setup the environment

In [1]:
import dotenv
import os
import logfire

import tool_functions as tf

import nest_asyncio

nest_asyncio.apply()

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = os.getenv("DEFAULT_MODEL")

# configure logfire
logfire.configure(token="pylf_v1_eu_jBBZFhCgN5s0hYgrFYK0wtbV5zt1Z9hhBM11nTXf6vlW")
logfire.instrument_pydantic_ai()

Logfire project URL: ]8;id=828410;https://logfire-eu.pydantic.dev/elbrink/claude-test\https://logfire-eu.pydantic.dev/elbrink/claude-test]8;;\

1 Definieer een basis agent

In [2]:
from pydantic_ai import Agent
from pydantic_ai.models.anthropic import AnthropicModel

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = AnthropicModel(os.getenv("DEFAULT_MODEL"))

basic_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
)

Stel een vraag aan de agent

In [3]:
result = basic_agent.run_sync("geef een defintie van een agent")
print(result.output)

17:28:34.488 basic_agent run
17:28:34.489   chat claude-3-haiku-20240307
Een agent is een autonome entiteit die waarneemt en handelt in een omgeving om een bepaald doel te bereiken. De belangrijkste kenmerken van een agent zijn:

1. Autonomie: Een agent kan zelfstandig beslissingen nemen en acties ondernemen zonder direct menselijk ingrijpen.

2. Waarneming: Een agent neemt informatie waar uit zijn omgeving via sensoren of andere middelen.

3. Besluitvorming: Op basis van de waargenomen informatie kan de agent beslissingen nemen over de te ondernemen acties.

4. Doelgerichtheid: De agent streeft naar het bereiken van een bepaald doel of set van doelen.

5. Adaptiviteit: Een agent kan zich aanpassen aan veranderingen in zijn omgeving en zijn gedrag bijstellen om zijn doelen te bereiken.

Agenten kunnen in verschillende vormen voorkomen, zoals softwareagenten, robotagenten of zelfs menselijke agenten. De kern is dat ze autonoom kunnen opereren om hun doel te bereiken in een dynamische om

Definieer tools voor de agent

In [4]:
tool_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
)


@tool_agent.tool_plain
def tool_bereken_btw(bedrag: float, btw_percentage: float):
    """
    Bereken BTW over een bedrag.
    """
    return tf.calculate_btw(bedrag, btw_percentage)


@tool_agent.tool_plain
def tool_bereken_korting(bedrag: float, korting_percentage: float):
    """
    Bereken korting over een bedrag.
    """
    return tf.calculate_discount(bedrag, korting_percentage)


@tool_agent.tool_plain
def tool_get_inwoners_gemeente(gemeente_naam: str, jaar: str):
    """
    Haal inwonersaantallen op van Nederlandse gemeenten via CBS API.
    """
    return tf.get_inwoners_gemeente(gemeente_naam, jaar)

Test een aantal tool calls

In [5]:
result2 = tool_agent.run_sync(
    "ik koop een product X voor 100 euro exclusief BTW, bereken de BTW. Ik krijg van de leverancier ook nog 15% korting op het bedrag inclusief BTW. Geef een overzicht wat ik moet betalen en hoe dat is samengesteld"
)
print(result2.output)

17:28:36.847 tool_agent run
17:28:36.848   chat claude-3-haiku-20240307
17:28:38.061   running 1 tool
17:28:38.061     running tool: tool_bereken_btw
17:28:38.063   chat claude-3-haiku-20240307
17:28:39.473   running 1 tool
17:28:39.474     running tool: tool_bereken_korting
17:28:39.476   chat claude-3-haiku-20240307
Met een korting van 15% op het bedrag inclusief BTW (€121) komt daar €18,15 korting vanaf, waardoor je uiteindelijk €102,85 moet betalen.

Samengevat:
- Oorspronkelijk bedrag excl. BTW: €100
- BTW (21%): €21
- Totaal bedrag incl. BTW: €121
- Korting (15%): €18,15
- Eindprijs: €102,85


In [6]:
result = tool_agent.run_sync(
    "Geef een overzicht van de top-7 gemeenten in Nederland met het grootste aantal inwoners in 2024. Laat ook het aantal inwoners van Groningen zien."
)
print(result.output)

17:28:40.950 tool_agent run
17:28:40.951   chat claude-3-haiku-20240307
17:28:42.250   running 1 tool
17:28:42.250     running tool: tool_get_inwoners_gemeente
17:28:42.877   chat claude-3-haiku-20240307
17:28:44.035   running 1 tool
17:28:44.036     running tool: tool_get_inwoners_gemeente
17:28:44.612   chat claude-3-haiku-20240307
17:28:45.643   running 1 tool
17:28:45.644     running tool: tool_get_inwoners_gemeente
17:28:46.257   chat claude-3-haiku-20240307
17:28:47.332   running 1 tool
17:28:47.333     running tool: tool_get_inwoners_gemeente
17:28:47.773   chat claude-3-haiku-20240307
17:28:49.047   running 1 tool
17:28:49.047     running tool: tool_get_inwoners_gemeente
17:28:49.521   chat claude-3-haiku-20240307
17:28:50.861   running 1 tool
17:28:50.862     running tool: tool_get_inwoners_gemeente
17:28:51.392   chat claude-3-haiku-20240307
17:28:53.207   running 1 tool
17:28:53.208     running tool: tool_get_inwoners_gemeente
17:28:53.728   chat claude-3-haiku-20240307
De t

In [7]:
result = tool_agent.run_sync(
    "Als alle inwoners uit Eindhoven in 2025 15 euro betalen, hoeveel krijg ik dan."
)
print(result.output)

17:28:55.123 tool_agent run
17:28:55.124   chat claude-3-haiku-20240307
17:28:56.233   running 1 tool
17:28:56.234     running tool: tool_get_inwoners_gemeente
17:28:56.741   chat claude-3-haiku-20240307
Volgens de gegevens had Eindhoven in 2025 ongeveer 249.054 inwoners.

Als elke inwoner 15 euro betaalt, dan is de totale opbrengst:
249.054 x 15 euro = 3.735.810 euro

Dus als alle inwoners van Eindhoven in 2025 elk 15 euro betalen, dan krijg je in totaal 3.735.810 euro.


Voeg geheugen toe aan de agent

In [8]:
mem_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
)

result = mem_agent.run_sync(
    "wat was ook alweer het btw bedrag van het eerder gekochte product X en hoeveel korting kreeg ik. Zoek dit uit de conversatie hiervoor.",
    message_history=result2.all_messages(),
)
print(result.output)

17:28:58.384 mem_agent run
17:28:58.384   chat claude-3-haiku-20240307
Okay, laten we dat even terugvinden:

Uit de eerdere berekeningen blijkt:

- Het bedrag exclusief BTW voor product X was: €100
- Het BTW bedrag was: €21
- De korting die je kreeg was: 15% op het bedrag inclusief BTW (€121)

Dus de BTW van het product X was €21 en je kreeg 15% korting op het bedrag inclusief BTW.
